In [ ]:
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
%matplotlib inline

image_all = mpimg.imread("todo.png",True)
_ = plt.figure(figsize=(40,80))
_ = plt.imshow(image_all, cmap='gray')


In [ ]:
# convetimos la imagen a blanco y negro (1bit)
intensity_thres = 70
image_bw = (image_all>intensity_thres)

#### Busqueda de puntos de corte horizontales y verticales
Para ello se suma la imagen en vertical y en horizontal. Donde están las rayas negras son mínimos. Se detectan esos mínimos para cortar

In [ ]:
# Suma de la imagen en blanco y negro para detectar las rayas horizontales
horizontal = image_bw.sum(1)
thrsH = 500

n = 5000
_ = plt.figure(figsize=(20,10))
_ = plt.plot(range(n), horizontal[0:n], range(n), [thrsH]*n)


In [ ]:
# Suma de la imagen en blanco y negro para detectar las rayas horizontales

vertical = image_bw.sum(0)
thrsV = 5000
n = len(vertical)
_ = plt.plot(range(n), vertical, range(n), [thrsV]*n)

In [ ]:
# Cortes
def cortes(valores, umbral, crop=15):
    """ A partir de un vector de valores y un umbral busca 
        donde la curva sube y luego baja. Entre medias esta
        la imagen que buscamos. Luego reduce los valores 
        encontrados con 'crop' para no seleccionar las rayas 
        negras"""
    zonas_picos = valores>umbral
    cuts = []
    i1 = 0
    while True:
        if i1 >= len(zonas_picos)-1:
            break
        if zonas_picos[i1] == False and zonas_picos[i1+1] == True:
            for i2 in range(i1+1, len(zonas_picos)-1):
                if zonas_picos[i2] == True and zonas_picos[i2+1] == False:
                    cuts.append((i1+crop, i2-crop))
                    i1 = i2
                    break
                    
        i1 = i1 + 1

    return list(cuts)

cutsH = cortes(horizontal, thrsH, 21)
cutsV = cortes(vertical, thrsV, 15)
print("Se han encontrado {0} filas y {1} columnas".format(len(cutsH), len(cutsV)))

In [ ]:

class Numerito:                                                               
    def __init__(self, ancho, alto):
        self.clase = -1
        self.imagen = np.full((int(ancho),int(alto)), 0)
        

#cutsH = np.linspace(51,57310,121)
#cutsH = [(int(round(v1)),int(round(v2))) for v1,v2 in zip(np.linspace(56,57315,121), np.linspace(451,57710,121))]


In [ ]:
from scipy import misc

# Corta las imágenes de los números y las guarda las imágenes
# También se guardan en la lista 'numeritos'

# Calculamos los máximos de ancho y alto para guardar todas las 
# imágenes de un mismo tamaño
w = max(map(lambda c:c[1]-c[0],cutsV))
h = max(map(lambda c:c[1]-c[0],cutsH))
w = w+1 if w%2==1 else w
h = h+1 if h%2==1 else h
ww = w
hh = h

i = 0
numeritos = []
for ch in cutsH:
    for cv in cutsV:
        num = Numerito(hh, ww)
        
        # Extraemos la imagen y se pega en el centro de la imagen de num
        im = image_all[ch[0]:ch[1],cv[0]:cv[1]]
        dy,dx = im.shape[0]/2,im.shape[1]/2 
        num.imagen[int(round(hh/2-dy)):int(round(hh/2-dy)+(ch[1]-ch[0])),
                   int(round(ww/2-dx)):int(round(ww/2-dx)+(cv[1]-cv[0]))] = im
        
        num.clase = i % 10
        
        numeritos.append(num)
 
        misc.toimage(num.imagen).save('nums/i%05d.png' % (i))
    
        i += 1
        

In [ ]:
     
# Se saca por pantalla una muestra de las imágenes
plt.figure(figsize=(10,10))
for i,iimg in enumerate(np.random.randint(0,len(numeritos), size=100)):
    plt.subplot(10,10,i+1)
    plt.imshow(numeritos[iimg].imagen)
        